In [ ]:
# Import libraries
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
import random
import json
import os

In [ ]:
def load_data_with_columns(input_path, num_simulations, num_increments, num_features):
    """
    Load data while ignoring the first four columns, then reshape for LSTM input.

    Args:
    - input_path (str): Path to the data file.
    - num_simulations (int): Number of simulations in the dataset.
    - num_increments (int): Number of increments per simulation.
    - num_features (int): Number of relevant features after.

    Returns:
    - reshaped_data (np.ndarray): Data reshaped to (num_simulations, num_increments, num_features).
    """
    data = pd.read_csv(input_path, header=None)

    data = data.iloc[:, 0:num_features]
    
    # Convert to numpy and reshape
    data_np = data.to_numpy()
    reshaped_data = data_np.reshape(num_simulations, num_increments, num_features)

    return reshaped_data

In [ ]:
input_path = 'path_to_input_file'
target_path = 'path_to_target_file'

In [ ]:
# Define parameters
num_simulations, num_increments, num_features, num_targets = 863, 1000, 12, 6
feature_data = load_data_with_columns(input_path, num_simulations, num_increments, num_features)
target_data = load_data_with_columns(target_path, num_simulations, num_increments, num_targets)

In [ ]:
def manual_minmax_scaler(data, feature_range=(-1, 1)):
    """
    Min-max scaler that computes data_min, data_max, and scale for each feature or target independently.
    """
    # Compute min and max for each feature/target across all simulations and time steps
    data_min = np.min(data, axis=(0, 1))  # Minimum for each feature/target
    data_max = np.max(data, axis=(0, 1))  # Maximum for each feature/target
    data_range = data_max - data_min
    scale = np.where(data_range != 0, (feature_range[1] - feature_range[0]) / data_range, 0)

    # Scale data for each feature/target
    data_scaled = np.where(data_range != 0, scale * (data - data_min) + feature_range[0], feature_range[0])
    return data_scaled, data_min, data_max, scale

def manual_inverse_transform(scaled_data, data_min, data_max, scale, feature_range=(-1, 1)):
    return np.where(scale != 0, (scaled_data - feature_range[0]) / scale + data_min, data_min)

In [ ]:
def normalize_data(data, scaler_function=manual_minmax_scaler):
    """
    Normalize data for each feature/target independently.
    """
    normalized_data, data_min, data_max, scale = scaler_function(data)
    scaling_params = {'data_min': data_min, 'data_max': data_max, 'scale': scale}
    return normalized_data, scaling_params

In [ ]:
feature_data_normalized, feature_scaler = normalize_data(feature_data)
target_data_normalized, target_scaler = normalize_data(target_data)

In [ ]:
print(feature_scaler)

In [ ]:
class CustomLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Gates: input, forget, cell, and output
        self.W_i = nn.Linear(input_size, hidden_size)
        self.U_i = nn.Linear(hidden_size, hidden_size)
        self.W_f = nn.Linear(input_size, hidden_size)
        self.U_f = nn.Linear(hidden_size, hidden_size)
        self.W_c = nn.Linear(input_size, hidden_size)
        self.U_c = nn.Linear(hidden_size, hidden_size)
        self.W_o = nn.Linear(input_size, hidden_size)
        self.U_o = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h_prev, c_prev):
        # Input gate
        i_t = torch.sigmoid(self.W_i(x) + self.U_i(h_prev))
        #print(i_t.shape)
        # Forget gate
        f_t = torch.sigmoid(self.W_f(x) + self.U_f(h_prev))
        #print(f_t.shape)
        # Cell candidate
        c_hat_t = torch.tanh(self.W_c(x) + self.U_c(h_prev))
        #print(c_hat_t.shape)
        # Cell state
        c_t = f_t * c_prev + i_t * c_hat_t
        #print(c_t.shape)
        # Output gate
        o_t = torch.sigmoid(self.W_o(x) + self.U_o(h_prev))
        #print(o_t.shape)
        # Hidden state
        h_t = o_t * torch.tanh(c_t)
        #print(h_t.shape)

        return h_t, c_t

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim  # Can be an int or a list
        self.num_layers = num_layers
        self.output_dim = output_dim

        if isinstance(hidden_dim, int):
            self.hidden_dim = [hidden_dim] * num_layers

        self.lstm_cells = nn.ModuleList([
            CustomLSTMCell(
                input_size=input_dim if i == 0 else self.hidden_dim[i - 1],
                hidden_size=self.hidden_dim[i]
            ) for i in range(num_layers)
        ])

        self.fc = nn.Linear(self.hidden_dim[-1], output_dim)

    def forward(self, x_t, h_t, c_t):
        """Process a single timestep."""
        for i, lstm_cell in enumerate(self.lstm_cells):
            if i == 0:
                h_t[i], c_t[i] = lstm_cell(x_t, h_t[i], c_t[i])
            else:
                h_t[i], c_t[i] = lstm_cell(h_t[i - 1], h_t[i], c_t[i])
        output = self.fc(h_t[-1])
        return output, h_t, c_t

In [ ]:
train_features, test_features, train_targets, test_targets = train_test_split(
    feature_data_normalized, target_data_normalized, test_size=0.1, random_state=42)
batch_size = 100
train_dataset = TensorDataset(torch.Tensor(train_features), torch.Tensor(train_targets))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(torch.Tensor(test_features), torch.Tensor(test_targets))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def train_custom_lstm_timestep(model, train_loader, num_epochs, learning_rate,loss_save_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.SmoothL1Loss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    epoch_losses = []  # List to store epoch losses
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            batch_size, seq_len, _ = inputs.size()
            h_t = [torch.zeros(batch_size, model.hidden_dim[i], device=device)
                   for i in range(model.num_layers)]
            c_t = [torch.zeros(batch_size, model.hidden_dim[i], device=device)
                   for i in range(model.num_layers)]

            optimizer.zero_grad()
            outputs = []

            # Process each timestep
            for t in range(seq_len):
                x_t = inputs[:, t, :]
                y_t = targets[:, t, :]
                output_t, h_t, c_t = model.forward(x_t, h_t, c_t)
                outputs.append(output_t.unsqueeze(1))

            outputs = torch.cat(outputs, dim=1)  # Shape: (batch_size, seq_len, output_dim)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_losses.append(epoch_loss)  # Save epoch loss
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.8f}")

    # Save epoch losses to a text file
    with open(loss_save_path, 'w') as f:
        for epoch, loss in enumerate(epoch_losses, 1):
            f.write(f"Epoch {epoch}: {loss:.8f}\n")

    print(f"Losses saved to {loss_save_path}")

    return model

In [ ]:
# Instantiate the model
model = LSTMModel(input_dim=12, hidden_dim=50, num_layers=2, output_dim=6)
# Train the model
loss_save_path = 'path_to_loss_file'
model = train_custom_lstm_timestep(model, train_loader, num_epochs=2, learning_rate=0.0002, loss_save_path=loss_save_path)

In [ ]:
save_path = 'path_to_save_model'
save_path_pt = 'path_to_save_scripted_model'
torch.save(model.state_dict(), save_path)
scripted_model = torch.jit.script(model)
scripted_model.save(save_path_pt)

In [ ]:
# Save function for scaling parameters
def save_scaling_params(scaler, filepath):
    """
    Save scaling parameters to a JSON file.

    Args:
    - scaler (dict): Dictionary with scaling parameters.
    - filepath (str): Path to the output JSON file.
    """
    # Convert numpy arrays to lists
    scaler = {key: value.tolist() if isinstance(value, np.ndarray) else value
                           for key, value in scaler.items()}
    
    with open(filepath, 'w') as f:
        json.dump(scaler, f)

# Save feature and target scalers
save_scaling_params(feature_scaler, 'path_to_feature_scaler')
save_scaling_params(target_scaler, 'path_to_target_scaler')

In [ ]:
# Function to convert JSON structure to arrays
def extract_scaling_params(json_file):
    """
    Extract scaling parameters from a JSON file.

    Args:
    - json_file (str): Path to the JSON file.

    Returns:
    - min_vals (np.ndarray): Array of minimum values for each feature/target.
    - max_vals (np.ndarray): Array of maximum values for each feature/target.
    """
    with open(json_file, "r") as f:
        scaling_params = json.load(f)
    min_vals = np.array(scaling_params['data_min'])
    max_vals = np.array(scaling_params['data_max'])
    return min_vals, max_vals

# Function to save arrays with comma-separated formatting, ensuring no line breaks
def save_array_with_commas(file_path, array):
    """
    Save an array to a file as a single comma-separated line.

    Args:
    - file_path (str): Path to the output text file.
    - array (np.ndarray): Array to save.
    """
    with open(file_path, "w") as f:
        # Write all elements in the array as a single comma-separated line
        array_str = ",".join(f"{val:.8e}" for val in array)
        f.write(array_str + "\n")

# Process feature scaling parameters
feature_min_vals, feature_max_vals = extract_scaling_params(
    "path_to_feature_scaler"
)
save_array_with_commas(
    "path_to_feature_min_vals",
    feature_min_vals,
)
save_array_with_commas(
    "path_to_feature_max_vals",
    feature_max_vals,
)

# Process target scaling parameters
target_min_vals, target_max_vals = extract_scaling_params(
    "path_to_target_scaler"
)
save_array_with_commas(
    "path_to_target_min_vals",
    target_min_vals,
)
save_array_with_commas(
    "path_to_target_max_vals",
    target_max_vals,
)

print("Scaling parameters saved as text files with commas, ensuring no line breaks.")


In [ ]:
def validate_and_plot_single_simulation(
    model, val_loader, target_scaler, timestep_mode=True, save_directory="."
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    predictions = []
    targets = []
    h_t_list = []
    c_t_list = []

    with torch.no_grad():
        for inputs, target in val_loader:
            inputs, target = inputs.to(device), target.to(device)
            batch_size, seq_len, input_dim = inputs.size()

            if timestep_mode:
                h_t = [torch.zeros(batch_size, model.hidden_dim[layer], device=device)
                       for layer in range(model.num_layers)]
                c_t = [torch.zeros(batch_size, model.hidden_dim[layer], device=device)
                       for layer in range(model.num_layers)]
                timestep_outputs = []

                for t in range(seq_len):
                    x_t = inputs[:, t, :]
                    output_t, h_t, c_t = model.forward(x_t, h_t, c_t)
                    timestep_outputs.append(output_t.unsqueeze(1))

                    h_t_list.append([h.detach().cpu().numpy() for h in h_t])
                    c_t_list.append([c.detach().cpu().numpy() for c in c_t])

                outputs = torch.cat(timestep_outputs, dim=1)
            else:
                outputs = model(inputs)

            predictions.append(outputs.cpu().numpy())
            targets.append(target.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    
    data_min = target_scaler['data_min']
    data_max = target_scaler['data_max']
    scale = target_scaler['scale']
    
    # Inverse scaling for predictions and targets
    predictions_original = manual_inverse_transform(predictions, data_min, data_max, scale, feature_range=(-1, 1))
    targets_original = manual_inverse_transform(targets, data_min, data_max, scale, feature_range=(-1, 1))
    
    
    
    # Scale only the first 3 features/targets
    #columns_to_scale = [0, 1, 2]
    #predictions_original = np.copy(predictions)
    #targets_original = np.copy(targets)

    #for col in columns_to_scale:
        #predictions_original[:, :, col] = manual_inverse_transform(
            #predictions[:, :, col], data_min[col], data_max[col], scale[col]
        #)
        #targets_original[:, :, col] = manual_inverse_transform(
            #targets[:, :, col], data_min[col], data_max[col], scale[col]
        #)

    # Random simulation index for plotting
    random_simulation_index = random.randint(0, predictions_original.shape[0] - 1)
    num_outputs = predictions_original.shape[2]
    time_increments = np.arange(predictions_original.shape[1])

    # Create folder for plots
    simulation_folder = os.path.join(save_directory, f"Simulation_V1_{random_simulation_index}")
    os.makedirs(simulation_folder, exist_ok=True)
    print(f"Saving plots to folder: {simulation_folder}")

    for i in range(num_outputs):
        plt.figure()
        plt.plot(time_increments, predictions_original[random_simulation_index, :, i],
                 label="Predicted", linestyle="-", marker="o")
        plt.plot(time_increments, targets_original[random_simulation_index, :, i],
                 label="True", linestyle="--", marker="x")
        plt.xlabel("Time Increments")
        plt.ylabel(f"Output {i + 1}")
        plt.title(f"Predicted vs True for Output {i + 1} - Simulation {random_simulation_index}")
        plt.legend()
        
        plt.gca().patch.set_facecolor('white')  # Set plot area background to white
        plt.gca().patch.set_edgecolor('black')  # Set edges of the plot area to black
        plt.gca().patch.set_linewidth(1)        # Set line width for the edges
        # Remove inner edge spines (contour inside the plot area)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_visible(False)
    
        # Save the plot and confirm
        plot_path = os.path.join(simulation_folder, f"output_{i + 1}.png")
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot: {plot_path}")

    print(f"All plots saved successfully in: {simulation_folder}")
    
    print(targets_original[random_simulation_index, :, :])
    
    if timestep_mode:
        return predictions_original, targets_original, h_t_list, c_t_list
    else:
        return predictions_original, targets_original

In [ ]:
model = LSTMModel(input_dim=12, hidden_dim=50, num_layers=2, output_dim=6)
model.load_state_dict(torch.load('path_to_save_model'))
model.eval()
validate_and_plot_single_simulation(model, val_loader, target_scaler, timestep_mode=True, save_directory='path_to_save_plots')